In [1]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

C:\MAD4AG


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm



import warnings

warnings.filterwarnings('ignore')

In [3]:
df_MAD = pd.read_parquet(f'./dbs/intermediate/indi_weights_distance.parquet')

In [4]:
df_survey_twins = pd.read_pickle(f'./dbs/intermediate/df_survey_twins.pkl')


# add region information

In [5]:
lan= pd.read_csv(f'./dbs/lan.csv', delimiter=';', dtype={'code':'string'})
lan.rename(columns={'code':'county', 'landsdelar':'region'},inplace=True)

df_MAD = pd.merge(df_MAD, lan[['county','region']], on='county')
df_survey_twins = pd.merge(df_survey_twins, lan[['county','region']], on='county')


# calculate median distance

In [6]:
avg_dist_threshold = df_survey_twins.avg_dist.median()
com_dist_threshold = df_survey_twins.com_distance.median()


# create stratum

In [7]:
df_MAD['region'] = df_MAD['region'].replace({'Svealand':'0','Göteland':'1', 'Norrland':'2'})

df_MAD['urban_density'] = df_MAD['urban_density'].replace({'A': '0', 'B': '1', 'C': '1'})

df_MAD['avg_dist_cat'] = '0'
df_MAD['avg_dist_cat'][df_MAD.avg_dist > avg_dist_threshold] = '1'

df_MAD['com_dist_cat'] = '0'
df_MAD['com_dist_cat'][df_MAD.com_dist > com_dist_threshold] = '1'

df_MAD.rename(columns={'commute':'employee'}, inplace=True)

In [8]:

df_MAD['stratum'] = df_MAD['region'].astype(str) + df_MAD['urban_density'].astype(str) +  df_MAD['employee'].astype(str)+  df_MAD['avg_dist_cat'].astype(str)

df_MAD['stratum'][(df_MAD.employee == 1)] = df_MAD['stratum'][(df_MAD.employee == 1)] + df_MAD['com_dist_cat'][(df_MAD.employee == 1)].astype(str)

df_MAD['stratum'].value_counts()

0101     51023
0100     38404
1101     35680
1100     33939
2100     12006
11100    11432
01100    10035
01111     9021
11111     8482
2101      7370
1001      6483
0001      5096
01110     3985
21100     3791
11110     3599
1000      3290
10111     2572
2001      2420
0000      1948
00111     1561
21111     1545
11101     1488
01101     1382
2000      1364
21110      916
20111      870
10100      771
10110      719
00110      437
00100      416
10101      284
21101      275
20100      258
20110      255
00101      144
20101       91
Name: stratum, dtype: int64

In [9]:
df_survey_twins['region'] = df_survey_twins['region'].replace({'Svealand':'0','Göteland':'1', 'Norrland':'2'})

df_survey_twins['urban_density'] = df_survey_twins['urban_density'].replace({'A': '0', 'B': '1', 'C': '1'})

df_survey_twins['avg_dist_cat'] = '0'
df_survey_twins['avg_dist_cat'][df_survey_twins.avg_dist > avg_dist_threshold] = '1'

df_survey_twins['com_dist_cat'] = '0'
df_survey_twins['com_dist_cat'][df_survey_twins.com_distance > com_dist_threshold] = '1'


In [10]:

df_survey_twins['stratum'] = df_survey_twins['region'].astype(str) + df_survey_twins['urban_density'].astype(str) +  df_survey_twins['employee'].astype(str)+  df_survey_twins['avg_dist_cat'].astype(str)

df_survey_twins['stratum'][(df_survey_twins.employee == 1)] = df_survey_twins['stratum'][(df_survey_twins.employee == 1)] + df_survey_twins['com_dist_cat'][(df_survey_twins.employee == 1)].astype(str)

df_survey_twins['stratum'].value_counts()

11100    2753
11111    2314
1100     1445
01100    1230
01111     993
11110     856
1101      705
10111     683
0100      616
11101     474
01110     339
21100     317
1001      293
0101      254
10110     231
00111     188
01101     187
2100      143
10100     118
20111     111
21111     105
0001       94
1000       75
2001       73
00110      64
21110      62
2101       47
10101      30
20110      26
21101      24
00100      12
0000        8
00101       4
20100       3
Name: stratum, dtype: int64

In [11]:
list_survey_type = list(df_survey_twins['stratum'].unique())
list_MAD_type = list(df_MAD['stratum'].unique())


set(list_MAD_type).difference(set(list_survey_type))

{'2000', '20101'}

In [12]:
threshold_count = 50

df_stratum = pd.DataFrame(columns=['stratum','level','mad_size','sample_size' , 'sur_size','survey_ppl'])


for i in list_MAD_type:
    df_MAD_p_type = df_MAD[df_MAD['stratum']==i]
    #print('mad len: ', len(df_MAD_p_type))

    df_sur_p_type = df_survey_twins[df_survey_twins['region']==i[0]]

    df_sur_temp = df_sur_p_type
    level = 0
    if len(df_sur_p_type) > threshold_count:
        df_sur_temp = df_sur_p_type[df_sur_p_type['urban_density']==i[1]]
        level = 1
        if len(df_sur_temp) > threshold_count:
            df_sur_p_type = df_sur_temp
            df_sur_temp = df_sur_p_type[df_sur_p_type['employee']==int(i[2])]
            level = 2
            #print(level)
            if len(df_sur_temp) > threshold_count:
                df_sur_p_type = df_sur_temp
                df_sur_temp = df_sur_p_type[df_sur_p_type['avg_dist_cat']==i[3]]
                level = 3
                #print(int(i[2]))
                if (len(df_sur_temp) > threshold_count)&(int(i[2])==0):
                   df_sur_p_type = df_sur_temp
                   level = 4

                elif (len(df_sur_temp) > threshold_count)&(int(i[2])==1):
                    df_sur_p_type = df_sur_temp
                    df_sur_temp = df_sur_p_type[df_sur_p_type['com_dist_cat']==i[4]]
                    level = 4
                    #print(level)

                    if len(df_sur_temp) > threshold_count:
                        df_sur_p_type = df_sur_temp
                        level = 5

    df_stratum = df_stratum.append(dict(stratum=i, level=level, mad_size=df_MAD_p_type.wt.sum(), sample_size=len(df_MAD_p_type), sur_size=len(df_sur_p_type), survey_ppl=df_sur_p_type['sub_id'].tolist()), ignore_index=True)



## calculate activity seq distribution and work activity participation distribution

In [13]:
seq_dist = df_survey_twins.act_seq_simp.value_counts(normalize=True).reset_index(name='count_desired')
seq_dist['count_desired']=seq_dist['count_desired']*df_stratum.mad_size.sum()

In [14]:
act_par_dist = df_survey_twins.groupby(['employee','commute']).size().reset_index(name='count_desired')

act_par_dist['count_desired']= act_par_dist['count_desired']/act_par_dist['count_desired'].sum()

act_par_dist['count_desired']=act_par_dist['count_desired']*df_stratum.mad_size.sum()

## generate candidates df by stratums

In [15]:
df_candidates = pd.DataFrame(columns=['sub_id','employee','commute', 'act_seq', 'act_seq_simp','stratum','mad_size' ,'ratio'])


for index, row in df_stratum.iterrows():

    df_temp= df_survey_twins[['sub_id','employee','commute', 'act_seq', 'act_seq_simp']][df_survey_twins.sub_id.isin(row['survey_ppl'])]
    df_temp['stratum']= row['stratum']
    df_temp['mad_size']=row['mad_size']
    df_temp['ratio']=row['mad_size']/row['sur_size']
    df_candidates = df_candidates.append(df_temp)
    #print(row['survey_ppl'])
    #len(df_survey_twins[df_survey_twins.sub_id.isin(row['survey_ppl'])])



In [16]:
df_candidates.ratio.sum()

7683949.229684818

## fitting probabilities to the simplified activity seq distribution

In [17]:
for i in range(50):


    ## fit to candidates ratios to the act par distribution
    temp = df_candidates.groupby(['employee','commute'])['ratio'].sum().reset_index(name='count')
    merged_df = pd.merge(act_par_dist, temp, on=['employee','commute'], how='inner')
    merged_df['result_column'] = merged_df['count_desired'] / merged_df['count']
    df_candidates = pd.merge(df_candidates, merged_df[['employee','commute', 'result_column']], on=['employee','commute'])

    df_candidates['ratio']= df_candidates['ratio']*df_candidates['result_column']
    df_candidates.drop(columns='result_column', inplace=True)


    ## fit to candidates ratios to the mad size by stratum
    temp = df_candidates.groupby('stratum')['ratio'].sum().reset_index(name='count')
    df_candidates = pd.merge(df_candidates, temp, on='stratum')

    df_candidates['ratio'] = df_candidates['ratio'] * (df_candidates['mad_size'] / df_candidates['count'])
    df_candidates.drop(columns='count', inplace=True)

    ####################################################


    #fit to candidates ratios to the act seq distribution
    temp = df_candidates.groupby('act_seq_simp')['ratio'].sum().reset_index(name='count')
    merged_df = pd.merge(seq_dist, temp, left_on='index', right_on='act_seq_simp', how='inner')
    merged_df['result_column'] = merged_df['count_desired'] / merged_df['count']
    df_candidates = pd.merge(df_candidates, merged_df[['act_seq_simp', 'result_column']], on='act_seq_simp')

    df_candidates['ratio']= df_candidates['ratio']*df_candidates['result_column']
    df_candidates.drop(columns='result_column', inplace=True)

    ##here tone down very high probabilities
    # w0 = ((np.std(df_candidates.loc[:, 'ratio']) / np.mean(df_candidates.loc[:, 'ratio'])) ** 2 + 1) ** 0.5 * 3.5 * np.median( df_candidates.loc[:, 'ratio'])
    # df_candidates.loc[df_candidates['ratio'] > w0, 'ratio'] = w0

    #fit to candidates ratios to the mad size by stratum
    temp = df_candidates.groupby('stratum')['ratio'].sum().reset_index(name='count')
    df_candidates = pd.merge(df_candidates, temp, on='stratum')

    df_candidates['ratio'] = df_candidates['ratio'] * (df_candidates['mad_size'] / df_candidates['count'])
    df_candidates.drop(columns='count', inplace=True)





## sampling survey ppl by the calculated probabilities for each stratum

In [18]:
df_candidates['probability'] = df_candidates['ratio']/df_candidates['mad_size']

#df_MAD['twin_ppl'] =0

In [19]:
df_candidates_prob= df_candidates.groupby('stratum')['probability'].apply(list).reset_index(name='probability_list')


df_candidates_ppl= df_candidates.groupby('stratum')['sub_id'].apply(list).reset_index(name='ppl_list')

In [20]:
df_stratum = pd.merge(df_stratum, df_candidates_prob)

df_stratum = pd.merge(df_stratum, df_candidates_ppl)


In [21]:
df_stratum['twin_ppl'] = df_stratum.apply( lambda row: np.argmax(np.random.multinomial(1, row['probability_list'], row['sample_size']), axis=1).tolist()  , axis=1)

## matching sampled survey ppl with MAD ppl

In [22]:
df_survey_twins_new = df_survey_twins.set_index('sub_id')
df_survey_twins_new = df_survey_twins_new[['act_seq', 'act_seq_simp', 'avg_dist','commute']]

In [23]:
from collections import Counter


def other_count(data):
    act_tuple = data['act_seq_simp']
    other_count_seq = Counter(act_tuple).get('Other')
    if other_count_seq is not None:
        data['other_count_seq'] = other_count_seq
    else:
        data['other_count_seq'] = 0
    return data

In [24]:

tqdm.pandas()
df_survey_twins_new = df_survey_twins_new.apply(lambda row: other_count(row), axis=1)


In [25]:
# df_twins = pd.DataFrame()
#
# for index, row in df_stratum.iterrows():
#     twin_ppl_list = df_stratum.loc[index,'ppl_list']
#     twin_ppl_place = df_stratum.loc[index,'twin_ppl']
#     twin_ppl_list = [twin_ppl_list[place] for place in twin_ppl_place]
#
#     df_survey_candidates= df_survey_twins_new.loc[twin_ppl_list].reset_index()
#
#     df_survey_candidates.sort_values(by=['other_count_seq','avg_dist'], inplace=True)
#     df_survey_candidates.reset_index(inplace=True)
#     df_survey_candidates.drop(columns='index', inplace=True)
#
#     df_MAD_new = df_MAD[df_MAD.stratum==row['stratum']]
#     df_MAD_new.sort_values(by=['other_count','avg_dist'], inplace=True)
#     df_MAD_new.reset_index(inplace=True)
#     df_MAD_new.drop(columns='index', inplace=True)
# #
#     df_MAD_new = pd.concat([df_MAD_new, df_survey_candidates], axis=1, ignore_index=False)
#     df_twins = df_twins.append(df_MAD_new)




In [26]:
df_twins = pd.DataFrame()

for index, row in df_stratum.iterrows():
    twin_ppl_list = df_stratum.loc[index,'ppl_list']
    twin_ppl_place = df_stratum.loc[index,'twin_ppl']
    twin_ppl_list = [twin_ppl_list[place] for place in twin_ppl_place]

    df_survey_candidates= df_survey_twins_new.loc[twin_ppl_list].reset_index()
    # if str(row['stratum'])[2]==str(1):
    #     print(row['stratum'], df_survey_candidates.commute.sum()/len(df_survey_candidates))

    df_MAD_new = df_MAD[df_MAD.stratum==row['stratum']]

    zero_MAD = df_MAD_new[df_MAD_new.other_count == 0]
    zero_survey = df_survey_candidates[df_survey_candidates.other_count_seq == 0]

    #for i in range(len(df_MAD_new.other_count_seq.max())):
    if len(zero_MAD) <= len(zero_survey):

        zero_survey.sort_values(by=['avg_dist'], inplace=True)
        zero_survey.reset_index(inplace=True)
        zero_survey = zero_survey.iloc[:len(zero_MAD)]
        remain_survey = df_survey_candidates.drop(index=zero_survey.index.to_list())
        zero_survey.drop(columns='index', inplace=True)

        zero_MAD.sort_values(by=['avg_dist'], inplace=True)
        zero_MAD.reset_index(inplace=True)
        zero_MAD.drop(columns='index', inplace=True)

        zero_MAD = pd.concat([zero_MAD, zero_survey.iloc[:len(zero_MAD)]], axis=1, ignore_index=False)


    else:
        print(row['stratum'])

    df_twins = df_twins.append(zero_MAD)

    remain_MAD = df_MAD_new[df_MAD_new.other_count > 0]


    remain_survey.sort_values(by=['avg_dist'], inplace=True)
    remain_survey.reset_index(inplace=True)
    remain_survey.drop(columns='index', inplace=True)

    remain_MAD.sort_values(by=['avg_dist'], inplace=True)
    remain_MAD.reset_index(inplace=True)
    remain_MAD.drop(columns='index', inplace=True)

    remain_MAD = pd.concat([remain_MAD, remain_survey], axis=1, ignore_index=False)
    df_twins = df_twins.append(remain_MAD)

    #print(str(row['stratum'])[2])





In [30]:
filename = r'.\dbs\twins\run-evaluate_work.pkl'
%store -r filename


df_twins.to_pickle(filename)

## Evaluation

In [28]:
print('The share of the matched survey ppl: ', np.round(len(df_twins.sub_id.unique())/len(df_survey_twins.sub_id.unique()),2))

The share of the matched survey ppl:  0.98


In [29]:
# print('Repetition number of 10 most matched people \n', df_twins.sub_id.value_counts().head(10))